# Estimation from Samples

**CS5483 Data Warehousing and Data Mining**
___

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

%matplotlib widget
plt.rcParams["figure.figsize"] = (8, 4)

## Unbiased and Consistent Estimate

### Monte-Carlo Simulation

We will simulate the process of flipping a possibly biased coin using the module `numpy` (rename as `np`). (See an introduction [here](https://www.cs.cityu.edu.hk/~ccha23/cs1302book/Lecture9/Monte%20Carlo%20Simulation%20and%20Linear%20Algebra.html).)

In [ ]:
rng = np.random.default_rng(seed=0)

The above code sets a particular random seed for the simultation results to be reproducible.

The following chooses the probability $p$ of a coin coming up head. It uniformly randomly picks the value from the unit interval $[0,1)$.

In [ ]:
p = rng.random()  # randomly generate the probability of head.

To generate a sequence of random coin tosses:

In [ ]:
n = 5000
coin_tosses = rng.choice(["H", "T"], size=n, p=[p, 1 - p])
coin_tosses

The characters `'H'` and `'T'` denotes head and tail, respectively.

### M-Estimation

We can estimate $p$ from the empirical distribution of the coin tosses:

$$
\hat{p} := \frac1n \sum_{i=0}^{n-1} z_i,
$$

where $z_1,\dots,z_n$ are i.i.d. samples of

$$
Z := \begin{cases}
1 & \text{if head comes up}\\
0 & \text{if tail comes up.}
\end{cases}
$$

The formula is called 

- the sample average of $Z$ and 
- the M-estimate of the expectation $E[Z]$ of $Z$.

In [ ]:
(coin_tosses == "H").mean()

In the particular case $Z$ being the indicator of head coming up, the estimate $\hat{p}$ is the fraction of heads coming up in $n$ coin tosses.

**Exercise** The following proves that $\hat{p}$ is an unbiased estimate:

$$
\begin{aligned}
E[\hat{p}] &= E\left[\frac1n \sum_{i=0}^{n-1} z_i \right]\\
&= \frac1n \sum_{i=0}^{n-1} \underbrace{E[z_i]}_{\smash{=p}} && \text{by ???}\\
&= p.
\end{aligned}
$$

What is the missing reasoning?

YOUR ANSWER HERE

**Exercise** The following shows that $\hat{p}$ is a consistent estimate:

$$
\begin{aligned}\sigma &:=\sqrt{\operatorname{Var}[\hat{p}]}\\
&= \sqrt{\operatorname{Var}\left[\frac1{n} \sum_{i=0}^{n-1} z_i\right]}\\
&= \sqrt{\frac1{n^2} \operatorname{Var}\left[\sum_{i=0}^{n-1} z_i\right]}\\
&=\sqrt{\frac1{n^2} \sum_{i=1}^n \underbrace{\operatorname{Var}[z_i]}_{\smash{=p(1-p)}}} && \text{by ???}\\
&=\sqrt{\frac{p(1-p)}{n}}, \end{aligned}
$$

which goes to $0$ as $n$ goes to $\infty$.

What is the missing reasoning?

YOUR ANSWER HERE

By the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem), as $n$ goes to $\infty$, the estimate has a gaussian distribution almost surely.

In [ ]:
plt.figure(1, clear=True)
x = np.linspace(-4, 4, 8 * 10 + 1)
plt.plot(x, norm.pdf(x), color="red", label=r'$\frac{1}{\sqrt{2\pi}}e^{-x^2}$')
for i in range(3, 0, -1):
    plt.fill_between(
        x,
        norm.pdf(x),
        alpha=2 ** (-i),
        color="blue",
        label=rf"$\Pr(|\hat{{p}}-p|\leq{i}\sigma)\approx {(norm.cdf(i)-0.5)*200:.3g}\%$",
        where=(abs(x) <= i),
    )
plt.title(r"Standard normal distribution for $\frac{\hat{p}-p}{\sigma}$ as $n\to \infty$")
plt.xlabel(r"x")
plt.ylabel(r"probability density")
plt.legend()
plt.show()

Over $95\%$ of the time, the estimator falls within $2$ standard deviation away from the ground truth, i.e.,

$$
\Pr(\hat{p}\in [p-2\sigma, p+2\sigma]) \geq 0.95
$$

To illustrate the above, we will generate and plot the estimate $\hat{p}$ for different sample size $n$:

In [ ]:
size = 5000
n = np.arange(1, size + 1)
phat = (coin_tosses == "H").cumsum() / n  # use first n tosses to estimate
sigma = (p * (1 - p) / n) ** 0.5  # true standard deviations of the estimates

plt.figure(2, clear=True)
# plot the ground truth p
plt.axhline(p, color="red")

# fill the region 2 sigma away from p
plt.fill_between(
    n, p - 2 * sigma, p + 2 * sigma, color="red", alpha=0.2, label=r"$p\pm 2\sigma$"
)

# plot the estimates phat
plt.plot(n, phat, marker=".", color="blue", linestyle="", markersize=1)

# configure the plot
plt.ylim([0, 1])
plt.xlim([0, n.size])
plt.title(r"Plot of $\hat{p}$ vs sample size")
plt.xlabel("sample size")
plt.ylabel("probability")
plt.legend()
plt.show()

## Biased Estimate

To understand the concept of bias in estimation, imagine playing the coin-tossing game:

- You win if a coin toss comes up head.
- You get to choose 1 out of the $m$ coins $i\in \{0,\dots,m-1\}$ with unknown probability $p_i$ of coming up head.
- You can flip each coin $n$ times before making your choice.

**How to play the game?**

A particular strategy for playing the game is to 
1. estimate the chance $p_i$ by the empirical probability $\hat{p}_i$ for each coin $i$, and
1. select the coin (with ties broken arbitrarily)

  $$
  i^* := \arg\max_i \hat{p}_i.
  $$

It is easy to see that the chance of winning by the given strategy is $E[p_{i^*}]$. Is the strategy optimal? Can a player evaluate or estimate the chance of winning without knowing $p_i$'s? Is the following a good estimate:

$$
\hat{p}_{i^*} = \max_i\hat{p}_i
$$

```{note}
**How is the strategy related to data-mining?**

Suppose $\hat{p}_i$ is the empirical accuracy of the classifier $f_i$. A common model selection strategy is to 
- choose the classifier $i^*$ defined above that has the empirical accuracy, and
- estimate its performance by $\hat{p}_{i^*}$.
```

**How to evaluate the estimate?**

Consider the simple case $n=1$, $m=2$, and $p_0=p_1=0.5$. We have the following four equally likely events:

$$
\begin{array}{ccc} \hat{p}_0 & \hat{p}_1 & \max_i \hat{p}_i \\\hline
0 & 0 & 0\\ 
0 & 1 & 1\\ 
1 & 0 & 1\\ 
1 & 1 & 1\\ 
\end{array}
$$

**Exercise** For the above simple case, compute $E[p_{i^*}]$ and $E[\max_i\hat{p}_i]$. Is $\max_i\hat{p}_i$ an unbiased estimate of $E[p_{i^*}]$?

YOUR ANSWER HERE

Instead of carrying out the exact analysis, which involves [order statistics][os], we will conduct the [Monte-Carlo simulation][MC] of the coin tossing game. The simulation can be verified by hand-calculating the closed-form solution for $n=1$, $m=2$, and $p_0=p_1=0.5$.

[MC]: https://www.cs.cityu.edu.hk/~ccha23/cs1302book/Lecture9/Monte%20Carlo%20Simulation%20and%20Linear%20Algebra.html
[os]: https://en.wikipedia.org/wiki/Order_statistic

The following initializes the list `p_list` of probabilities of head for different coins:

In [ ]:
m = 2
p_list = np.array([0.4] * (m - 1) + [0.6])
# To generate the probability randomly instead, use
# p_list = rng.random(m)
p_list

Instead of generating a sequence of coin tosses, we will simulate $\hat{p}_i$ directly using the binomial distribution since

$$
n\hat{p}_i\sim \operatorname{Binomial}(n,p_i).
$$

In [ ]:
size = 10
n = np.arange(1, size + 1)
k = 100000
phat = rng.binomial(
    n.reshape(-1, 1, 1), p_list.reshape(1, -1, 1), (size, m, k)
) / n.reshape(-1, 1, 1)
max_phat = phat.max(axis=1)
max_phat

`max_phat` is a 2-dimensional array of samples of $\max_{i}\hat{p}_i$:
- The first axis indexes samples obtained from different number of tosses.
- The second axis indexes `k` independent samples for the same number of tosses.

The `k` independent samples can be used to approximates $E[\max_{i}\hat{p}_i]$ as follows.

In [ ]:
E_max_phat = max_phat.mean(axis=-1)
E_max_phat

Similarly, the winning probability can be approximated as follows:

In [ ]:
win_prob = p_list[phat.argmax(axis=1)].mean(axis=-1)
win_prob

The following plots compare the probabilities as a function of $n$.

In [ ]:
plt.figure(3, clear=True)
plt.axhline(p_list.max(), color="red", label=r"$\max_i p_i$")
plt.plot(
    n,
    E_max_phat,
    linestyle="--",
    marker=".",
    color="blue",
    markersize=10,
    label=r"$E[\max_i\hat{p}_i]$",
)
plt.plot(
    n,
    win_prob,
    linestyle=":",
    marker="x",
    color="green",
    markersize=10,
    label="winning probability",
)

plt.ylim([0, 1])
plt.xlim([n[0], n[-1]])
plt.title(r"Plot of $E[\max_i\hat{p}_i]$ vs $n$")
plt.xlabel("$n$")
plt.ylabel("probability")
plt.legend()
plt.show()

**Exercise** Compare the chance of winning with $\max_i p_i$ in general, for different $p_i$'s.

```{hint}

Change `p_list` to explore different the non-uniform cases where $p_i$'s may not be equal. Be careful about the deterministic case where $p_i\in \Set{0,1}$ for all $i$.

```

YOUR ANSWER HERE

**Exercise** Compare the chance of winning with $E[\max_i \hat{p}_i]$. Is $\max_i \hat{p}_i$ a biased estimate? If so, is it overly optimistic, i.e., has an expectation larger than the chance of winning?

YOUR ANSWER HERE

**Exercise** Is $\max_i \hat{p}_i$ a consistent estimate of the chance of winning?

YOUR ANSWER HERE